In [35]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
def read_ubyte_file(file_path):
    with open(file_path, 'rb') as f:
        # Read the magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        
        if magic_number == 2051:  # Magic number for images
            num_images = int.from_bytes(f.read(4), 'big')
            rows = int.from_bytes(f.read(4), 'big')
            cols = int.from_bytes(f.read(4), 'big')
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_images, rows, cols)
            return images
        
        elif magic_number == 2049:  # Magic number for labels
            num_labels = int.from_bytes(f.read(4), 'big')
            labels = np.frombuffer(f.read(), dtype=np.uint8)
            return labels
        
        else:
            raise ValueError('Invalid magic number in file header')

train_images_path = './data/train-images-idx3-ubyte'
train_labels_path = './data/train-labels-idx1-ubyte'

train_images = read_ubyte_file(train_images_path)
train_labels = read_ubyte_file(train_labels_path)


In [37]:
print(train_labels.shape)
print(train_labels[0])

(60000,)
5


In [38]:
print(train_images.shape)
# go from (60000, 28, 28) to (60000, 784)
X = train_images.reshape(train_images.shape[0],train_images.shape[1]*train_images.shape[2])
print(train_images.shape)


(60000, 28, 28)
(60000, 28, 28)


In [39]:
def init_params():
    # each pixel from image needs to point to 10 hidden neurons
    W1 = np.random.randn(784, 10) * 0.01 
    #1 bias for each of the 10 hidden neurons
    b1 = np.random.randn(1, 10) * 0.01   
    
    # each of the 10 hidden neurons needs to point to each of the 10 output neurons
    W2 = np.random.randn(10, 10) * 0.01  
    # 1 bias for each of the 10 output neurons
    b2 = np.random.randn(1, 10) * 0.01   
    
    return W1, b1, W2, b2

In [40]:
def ReLU(x):
    return np.maximum(0, x)

In [41]:
def soft_max(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

In [42]:
def one_hot_encode(labels, num_classes):
    # Initialize a matrix of zeros with shape (number of labels, number of classes)
    one_hot = np.zeros((labels.size, num_classes))
    
    # Set the corresponding index to 1 for each label
    one_hot[np.arange(labels.size), labels] = 1
    
    return one_hot

Y = one_hot_encode(train_labels, 10)

In [43]:
def compute_loss(A2, Y):
    m = Y.shape[0]
    Y_int = np.argmax(Y, axis=1)

    log_probs = -np.log(A2[np.arange(m), Y_int])
    loss = np.sum(log_probs) / m
    return loss

In [44]:
def forward(X, W1, b1, W2, b2, Y):
    Z1 = np.dot(X, W1) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = soft_max(Z2)
    return Z1, A1, Z2, A2

In [45]:
def back_prop(X, Y, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate):
    # used to find the normalized gradient
    # m = num of training examples in batch
    m = X.shape[0]
    dZ2 = A2 - Y
    
    # multiply the output of h1 by the change in the output
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    
    dA = np.dot(dZ2, W2.T)
    
    # relu derivative
    dZ1 = dA * (Z1 > 0)
    
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    return dW1, db1, dW2, db2

In [46]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [47]:
def train(X, Y, W1, b1, W2, b2, learning_rate, epochs):
    for i in range(epochs):
        # forward
        Z1, A1, Z2, A2 = forward(X, W1, b1, W2, b2, Y)
        loss = compute_loss(A2, Y)
        #backward
        dW1, db1, dW2, db2 = back_prop(X, Y, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate)
        
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        if i % 100 == 0:
            print(f'Epoch {i}: loss = {loss}')
    return W1, b1, W2, b2
        

In [48]:
W1, b1, W2, b2 = init_params()


W1, b1, W2, b2 = train(X, Y, W1, b1, W2, b2, 0.01, 1000)

Epoch 0: loss = 2.3807194896388113
Epoch 100: loss = 2.3025121226150183
Epoch 200: loss = 2.3022641159634705
Epoch 300: loss = 2.3020613366651577
